In [ ]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

In [25]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [26]:
# In-Memory Document Store
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [27]:
from google.colab import files
uploaded = files.upload()

Saving QG using t5 model.csv to QG using t5 model (1).csv


In [28]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['QG using t5 model.csv']))
df=df.drop('Unnamed: 0',axis=1)
df

,keyphrase,context,question
0,transfer learning,"['Transfer learning, where a model is first pre-trained on a data-rich task ...",What has emerged as a powerful technique in natural language processing?
1,natural language processing (nlp),"['Transfer learning, where a model is first pre-trained on a data-rich task ...",What did the paper introduce to explore the landscape of transfer learning t...
2,nlp,"['In this paper, we explore the landscape of transfer learning techniques fo...",What has the effectiveness of transfer learning given rise to?
3,text-to-text,"['In this paper, we explore the landscape of transfer learning techniques fo...",What do we do to facilitate future work on transfer learning for NLP?
4,language understanding,"['Our systematic study compares pretraining objectives, narchitectures, unla...",What do systematic studies compare on dozens of language understanding tasks?
5,pretraining objectives,"['Our systematic study compares pretraining objectives, narchitectures, unla...",What do systematic studies compare on dozens of language understanding tasks?
6,corpus,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?
7,summarization,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?
8,question answering,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?


In [29]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http
document_store.delete_all_documents()
docs = []
for i in df['context']:
  docs.append({'content':i})
print(docs[:])
# Now, let's write the docs to our DB.
document_store.write_documents(docs)

                1. delete_all_documents() method is deprecated, please use delete_documents method
                For more details, please refer to the issue: https://github.com/deepset-ai/haystack/issues/1045
                
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '38d44c154443032074664b80c667671' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'edbc99799fb9be68387e77036a3f0948' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'edbc99799fb9be68387e77036a3f0948' already exists in index 'document'


[{'content': "['Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned non a downstream task, has emerged as a powerful technique in natural language processing (NLP).', 'In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework nthat converts every language problem into a text-to-text format.', 'The effectiveness of transfer learning has given rise to a diversity of approaches, methodology, and practice.', 'To facilitate future work on transfer learning for NLP, nwe release our dataset, pre-trained models, and code.']"}, {'content': "['Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned non a downstream task, has emerged as a powerful technique in natural language processing (NLP).']"}, {'content': "['In this paper, we explore the landscape of transfer learning techniques for NLP by introducing a unified framework nthat converts every langua

In [30]:
from haystack.nodes import TfidfRetriever
retriever = TfidfRetriever(document_store=document_store)

INFO:haystack.nodes.retriever.sparse:Found 6 candidate paragraphs from 6 docs in DB


In [ ]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [32]:
from haystack.pipelines import ExtractiveQAPipeline
pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
answers = []
for i in df['question']:
  answers.append(pipe.run(query = i))

In [34]:
answers

[{'query': 'What has emerged as a powerful technique in natural language processing?',
  'no_ans_gap': 9.496152400970459,
  'answers': [<Answer {'answer': 'Transfer learning', 'type': 'extractive', 'score': 0.9725081920623779, 'context': "['Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned non a downstream task, has emerged as a powerful t", 'offsets_in_document': [{'start': 2, 'end': 19}], 'offsets_in_context': [{'start': 2, 'end': 19}], 'document_id': 'd739f09f9bab640e5cec953a5b98bd6a', 'meta': {}}>,
   <Answer {'answer': 'Transfer learning', 'type': 'extractive', 'score': 0.9364281296730042, 'context': "['Transfer learning, where a model is first pre-trained on a data-rich task before being fine-tuned non a downstream task, has emerged as a powerful t", 'offsets_in_document': [{'start': 2, 'end': 19}], 'offsets_in_context': [{'start': 2, 'end': 19}], 'document_id': '62577fb39bdcb637b47e823dabf6c45b', 'meta': {}}>,
   <Answer {'answer':

In [35]:
df['answers'] = answers
df

,keyphrase,context,question,answers
0,transfer learning,"['Transfer learning, where a model is first pre-trained on a data-rich task ...",What has emerged as a powerful technique in natural language processing?,{'query': 'What has emerged as a powerful technique in natural language proc...
1,natural language processing (nlp),"['Transfer learning, where a model is first pre-trained on a data-rich task ...",What did the paper introduce to explore the landscape of transfer learning t...,{'query': 'What did the paper introduce to explore the landscape of transfer...
2,nlp,"['In this paper, we explore the landscape of transfer learning techniques fo...",What has the effectiveness of transfer learning given rise to?,"{'query': 'What has the effectiveness of transfer learning given rise to?', ..."
3,text-to-text,"['In this paper, we explore the landscape of transfer learning techniques fo...",What do we do to facilitate future work on transfer learning for NLP?,{'query': 'What do we do to facilitate future work on transfer learning for ...
4,language understanding,"['Our systematic study compares pretraining objectives, narchitectures, unla...",What do systematic studies compare on dozens of language understanding tasks?,{'query': 'What do systematic studies compare on dozens of language understa...
5,pretraining objectives,"['Our systematic study compares pretraining objectives, narchitectures, unla...",What do systematic studies compare on dozens of language understanding tasks?,{'query': 'What do systematic studies compare on dozens of language understa...
6,corpus,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?,"{'query': 'What are some benchmarks nwe achieve?', 'no_ans_gap': 7.295330047..."
7,summarization,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?,"{'query': 'What are some benchmarks nwe achieve?', 'no_ans_gap': 7.295330047..."
8,question answering,['By combining the insights from our exploration with scale and our new “Col...,What are some benchmarks nwe achieve?,"{'query': 'What are some benchmarks nwe achieve?', 'no_ans_gap': 7.295330047..."
